In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用于在线预测的自定义训练图像超分辨率模型，具有预测后处理

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_super_resolution_online_post.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_super_resolution_online_post.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署一个自定义图像超分辨率模型进行在线预测，并对预测进行后处理。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已内置于TensorFlow中。训练模型会预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将学习如何使用Vertex客户端库在Docker容器中从Python脚本创建自定义模型，对部署的模型进行预测，并在服务二进制文件中进行后处理。您也可以使用`gcloud`命令行工具或在线使用Google Cloud控制台创建和部署模型。


执行的步骤包括：

- 为训练模型创建一个Vertex自定义作业。
- 使用自定义容器训练一个TensorFlow模型。
- 检索并加载模型文件。
- 查看模型评估。
- 构建用于后处理的服务函数。
- 上传模型作为一个Vertex `Model` 资源。
- 部署`Model`资源到一个服务`Endpoint`资源。
- 进行预测。
- 停止部署`Model`资源。

### 费用

本教程使用 Google Cloud (GCP) 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

## 安装

安装最新版本的 Vertex 客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新版本的*google-cloud-storage*库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

安装了 Vertex 客户端库和谷歌 *云存储* 后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有此选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您使用的是哪种笔记本环境，下面的步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud 笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!` 为前缀的行视为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您也可以更改`REGION`变量，该变量用于本笔记本的其余操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不会使用多区域存储桶来进行Vertex的训练。并非所有区域都支持所有的Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正处于实时教程会话中，您可能正在使用共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 账户

**如果您正在使用 Google Cloud Notebook**，您的环境已经经过身份验证。请跳过这一步。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**将此服务帐号授权给项目**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex 管理员**。在过滤框中输入"Storage Object Admin"，并选择**Storage Object Admin**。

点击创建。包含您密钥的 JSON 文件将下载到您的本地环境中。

在下面的单元格中输入您服务帐号密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

当您使用 Vertex 客户端库提交一个自定义的训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex会从这个包中运行代码。在本教程中，Vertex还会将训练作业产生的训练模型保存在相同的存储桶中。然后您可以基于这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有的 Google Cloud 项目中全局唯一，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查它的内容，验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、流水线和端点服务终点。
- `PARENT`：顶点数据集、模型、作业、流水线和端点资源的位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

设置用于训练和预测的硬件加速器（例如，GPU）。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持GPU的容器镜像和分配给虚拟机实例的GPU数量。例如，要使用一个具有 4 个 Nvidia Telsa K80 GPU 的容器镜像分配给每个虚拟机实例，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在 CPU 上运行的容器镜像。

*注意*: 在 2.3 之前的 TensorFlow 发行版中，GPU 支持将无法在本教程中加载自定义模型。这是一个已知问题，在 TensorFlow 2.3 中已修复 -- 这是由于生成在服务功能中的静态图操作造成的。如果您在自己的自定义模型上遇到此问题，请使用支持 GPU 的 TensorFlow 2.3 的容器镜像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将设置用于训练和预测的Docker容器镜像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU有3.75GB内存。
     - `n1-highmem`: 每个vCPU有6.5GB内存
     - `n1-highcpu`: 每个vCPU有0.9GB内存
 - `vCPUs`: 数目为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下类型不支持用于训练：*

 - `standard`: 2个vCPUs
 - `highcpu`: 2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已准备好开始为CIFAR10创建自己的定制模型和训练。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的一边（Python脚本）上，您需要创建一个客户端，用以向Vertex服务器发送请求并接收响应。

在本教程中，您将针对工作流程中的不同步骤使用不同的客户端。因此，请务必提前设置好它们。

- 用于`Model`资源的模型服务。
- 用于部署的终端服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供预测的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 训练模型

您可以使用容器镜像的两种方式来训练一个自定义模型:

- **使用Google Cloud预构建容器**。如果您使用预构建容器，您还需要指定一个要安装到容器镜像中的Python软件包。这个Python软件包包含了您用于训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果您使用您自己的容器，容器需要包含您用于训练自定义模型的代码。

准备您的定制工作规范

现在您的客户准备就绪，您的第一步是为您的定制培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec` : 用于训练的机器类型和数量规格（单个或分布式）
- `python_package_spec` : 与预构建容器一起安装的Python软件包规格

### 准备您的机器规格

现在为您的自定义训练作业定义机器规格。这告诉 Vertex 需要为训练提供哪种类型的机器实例。
 - `machine_type`：要提供的 GCP 实例类型 -- 例如，n1-standard-8。
 - `accelerator_type`：（如果有的话）硬件加速器的类型。在本教程中，如果您之前设置了变量 `TRAIN_GPU != None`，表示您正在使用 GPU；否则将使用 CPU。
 - `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉Vertex在训练过程中的每个机器实例上提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD更快，而标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小（以GB为单位）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

###定义工作池规范

接下来，您需要为自定义训练任务定义工作池规范。工作池规范将包括以下内容：

- `replica_count`: 要提供的此机器类型的实例数量。
- `machine_spec`: 硬件规格。
- `disk_spec` : （可选）磁盘存储规格。

- `python_package`: 要安装在VM实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们更深入地了解Python包规范：

- `executor_image_spec`: 这是为您的自定义训练任务配置的Docker镜像。

- `package_uris`: 这是要安装在提供的实例上的Python训练包的位置（URIs）列表。这些位置需要在Cloud Storage存储桶中。可以是单个Python文件，也可以是整个包的zip（存档）。在后一种情况下，作业服务将解压（解档）内容到Docker镜像中。

- `python_module`: 要调用以运行自定义训练任务的Python模块（脚本）。在本例中，您将调用 `trainer.task.py` -- 请注意，不需要附加`.py`后缀。

- `args`: 要传递给相应Python模块的命令行参数。在本例中，您将设置：
  - `"--model-dir=" + MODEL_DIR` : 存储模型工件的Cloud Storage位置。有两种方法可以告诉训练脚本在哪里保存模型工件：
      - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您将在作业规范中告诉服务模型工件位置。
  - `"--epochs=" + EPOCHS`: 训练的轮数。
  - `"--steps=" + STEPS`: 每轮的步数（批次）。
  - `"--distribute=" + TRAIN_STRATEGY"` : 用于单一或分布式训练的训练分发策略。
     - `"single"`: 单个设备。
     - `"mirror"`: 单个计算实例上的所有GPU设备。
     - `"multi"`: 所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 拼装工作规范

现在拼装自定义工作规范的完整描述：

- `display_name`：您为这个自定义工作分配的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器虚拟机实例的规范。
    - `base_output_directory`：这告诉服务在哪个云存储位置保存模型工件（变量`DIRECT = False`时）。然后服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将为：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训套件

#### 套件布局

在开始培训之前，您将查看如何为自定义培训任务组装 Python 套件。解压后，套件包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将套件安装到 Docker 镜像的操作环境中的指令。

文件 `trainer/task.py` 是用于执行自定义培训任务的 Python 脚本。*注意*，当我们在工作池规范中引用它时，我们会将目录斜杠替换为点号（`trainer.task`）并去掉文件后缀（`.py`）。

#### 套件组装

在下面的单元格中，您将组装培训套件。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image super resolution\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我们不会详细讨论，只是让您浏览一下。总结如下：

- 从命令行（`--model_dir`）获取保存模型产物的目录，如果未指定，则从环境变量 `AIP_MODEL_DIR` 获取。
- 从 TF Datasets（tfds）加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 使用参数 `args.epochs` 和 `args.steps` 对模型进行训练（`fit()`）。
- 将训练好的模型保存到指定模型目录（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储到您的云存储桶中

接下来，将培训文件夹打包成压缩的tar文件，然后存储到您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练工作。使用这个帮助函数 `create_custom_job`，它接受以下参数：

-`custom_job`: 自定义工作的规范。

帮助函数调用作业客户端服务的 `create_custom_job` 方法，具有以下参数：

-`parent`: Vertex 中 `Dataset`、`Model` 和 `Endpoint` 资源的位置路径。
-`custom_job`: 自定义工作的规范。

您将显示 `response` 对象中返回的一些字段，其中最感兴趣的两个字段是：

`response.name`: 分配给此自定义训练工作的Vertex完全限定标识符。您保存此标识符以在后续步骤中使用。

`response.state`: 自定义训练工作的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获得您创建的自定义工作的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关自定义作业的信息

接下来，请使用这个辅助函数`get_custom_job`，它需要以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的`get_custom_job`方法，使用以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

如果您还记得，在调用`create_custom_job`方法时，在`response.name`字段中获得了自定义作业的 Vertex 完全限定标识符，并将该标识符保存在变量`job_id`中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练以上模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所花费的实际时间。对于您的模型，我们需要知道保存模型的位置，即Python脚本保存在您本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载已保存的模型

您的模型以TensorFlow SavedModel格式存储在云存储桶中。现在从云存储桶中加载模型，然后您可以执行一些操作，如评估模型和进行预测。

要加载模型，您可以使用TF.Keras的`model.load_model()`方法，将保存模型的云存储路径（由`MODEL_DIR`指定）作为参数传递。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将使用`tf.keras.datasets`中的`load_data()`方法加载CIFAR10测试（留出）数据。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：图像数据和相应的标签。

您不需要训练数据，所以我们加载为`(_, _)`。

在可以通过评估数据之前，您需要对其进行预处理：

x_test:
1. 通过将每个像素除以255来对像素数据进行正规化（重新缩放）。这将用32位浮点数替换每个单字节整数像素，取值范围在0到1之间。

y_test:<br/>
2. 标签当前是标量（稀疏）。如果您回顾`trainer/task.py`脚本中的`compile()`步骤，会发现它是为稀疏标签编译的。因此我们无需做更多操作。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 进行模型评估

现在评估一下定制工作中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

上传模型以提供服务

接下来，您将把您的TF.Keras模型从自定义作业上传到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数来将数据转换为模型所需的格式。如果您将编码数据发送给Vertex，您的服务函数将确保在传递给模型输入之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，该请求将被一个HTTP服务器接收。HTTP服务器从HTTP请求内容主体中提取预测请求。提取的预测请求将被转发到服务函数。对于谷歌预构建的预测容器，请求内容将作为`tf.string`传递给服务函数。

服务函数包含两个部分：

- `预处理函数`：
  - 将输入（`tf.string`）转换为基础模型的输入形状和数据类型（动态图）。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，创建JSON对象等。

预处理和后处理函数都被转换为与模型融合的静态图。基础模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在构建用于TF.Keras模型的服务函数时，您需要考虑一个问题，即它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果这样做，您将在服务函数的编译过程中遇到错误，该错误将指示您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

#### 预处理

为了将图像传递给预测服务，您需要将压缩（例如，JPEG）图像字节编码为 base 64 格式 -- 这样可以使内容在通过网络传输时防止被修改。由于这个部署的模型期望输入数据为原始（未压缩）字节，您需要确保将 base 64 编码的数据转换回原始字节，然后将其作为输入传递给部署的模型。

为解决此问题，定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加 `@tf.function` 装饰器，以便服务函数与基础模型融合（而不是在 CPU 上游执行）。

当您发送预测或解释请求时，请求的内容将被 base 64 解码为一个 Tensorflow 字符串（`tf.string`），该字符串将传递给服务函数（`serving_fn`）。服务函数将预处理 `tf.string` 为原始（未压缩）的 numpy 字节（`preprocess_fn`），以匹配模型的输入要求：
- `io.decode_jpeg` - 对 JPG 图像进行解压，返回一个具有三个通道（RGB）的 Tensorflow 张量。
- `image.convert_image_dtype` - 将整数像素值转换为 float 32，并在 0 到 1 之间重新缩放像素数据。
- `image.resize` - 将图像大小调整为匹配模型的输入形状。

此时，数据可以传递给模型（`m_call`）。

#### 后处理

从 `prob = m_call(**images)` 返回的值将是一个张量列表，每个实例在预测请求中独立对应一个张量。每个张量将是对应实例的预测的超分辨率图像。

**待办事项**

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(16, 16))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 16, 16, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)


def _postprocess(bytes_output):
    rescale = tf.cast(bytes_output * 255.0, tf.uint8)
    reshape = tf.reshape(rescale, (32, 32, 3))
    encoded = tf.io.encode_jpeg(reshape)
    return tf.cast(encoded, tf.string)


@tf.function(input_signature=[tf.TensorSpec([None, 32, 32, 3], tf.float32)])
def postprocess_fn(bytes_outputs):
    encoded_images = tf.map_fn(
        _postprocess, bytes_outputs, dtype=tf.string, back_prop=False
    )
    return encoded_images


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    post = postprocess_fn(prob)
    return post


tf.saved_model.save(
    model,
    model_path_to_deploy,
    signatures={
        "serving_default": serving_fn,
    },
)

获取serving函数签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取您模型的输入和输出层的签名。

对于您的目的，您需要serving函数的签名。为什么呢？当我们将数据作为HTTP请求数据包发送以进行预测时，图像数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的serving函数将从base64转换为一个numpy数组。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称--在以后进行预测请求时会使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这一步，您可以像使用任何其他Vertex `Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源。

辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的人类可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们的SavedModel工件在云存储中的路径。对于本教程，这是`trainer/task.py`保存模型工件的云存储位置，我们在变量`MODEL_DIR`中指定。

辅助函数调用`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex位置根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个字典对象，包含以下字段：

- `display_name`：`Model`资源的人类可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex `Dataset`资源的情况下构建的，您将保持此处为空（`''`）。
- `artificat_uri`：以SavedModel格式存储模型的云存储路径。
- `container_spec`：这是要安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源将返回一个长时间运行的操作，因为可能需要一些时间。您调用response.result()，这是一个同步调用，在Vertex Model资源准备就绪时返回。

辅助函数返回对应的Vertex Model实例`upload_model_response.model`的Vertex完全限定标识符。您将保存此标识符以备后续步骤中使用。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

### 获取`Model`资源信息

现在让我们获取只有您的模型的模型信息。使用这个辅助函数`get_model`，带有以下参数：

- `name`：`Model`资源的 Vertex 唯一标识符。

这个辅助函数调用 Vertex `Model` 客户端服务的方法`get_model`，带有以下参数：

- `name`：`Model`资源的 Vertex 唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

## 部署 `Model` 资源

现在部署经过训练的 Vertex 定制 `Model` 资源。 这需要两个步骤：

1. 为部署 `Model` 资源创建一个 `Endpoint` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源中。

### 创建`Endpoint`资源

使用这个帮助函数`create_endpoint`来创建一个端点以部署模型以进行预测服务，需要以下参数：

- `display_name`：用于`Endpoint`资源的人类可读的名称。

该帮助函数使用端点客户端服务的`create_endpoint`方法，该方法需要以下参数：

- `display_name`：用于`Endpoint`资源的人类可读的名称。

创建一个`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来为服务预提供`Endpoint`资源。您调用`response.result()`，这是一个同步调用，并会在`Endpoint`资源准备就绪时返回。该帮助函数返回`Endpoint`资源的Vertex完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "cifar10_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“终端”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理在线预测请求时，您有几种调整计算实例规模的选择：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设定为一。

- 手动扩展：在线预测请求分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设定为相同数量的节点。当模型首次部署到实例上时，会预配固定数量的计算实例，并将在线预测请求均匀分配到这些实例上。

- 自动扩展：在线预测请求分配到可扩展数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）数量设定为首次部署模型时要预配和解除预配的实例数量，并根据负载情况将计算实例的最大（`MAX_NODES`）数量设定为要预配的实例数量。

在随后的部署请求中，最小计算实例数量对应于字段`min_replica_count`，最大计算实例数量对应于字段`max_replica_count`。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

使用这个辅助函数`deploy_model`将`Model`资源部署到您为提供预测创建的`Endpoint`资源，具体参数如下：

- `model`：要从训练流水线上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将模型部署到的 Vertex 完全限定终端点标识符。

辅助函数调用`Endpoint`客户端服务的`deploy_model`方法，该方法接受以下参数：

- `endpoint`：要将`Model`资源部署到的 Vertex 完全限定`Endpoint`资源标识符。
- `deployed_model`：部署模型的需求规范。
- `traffic_split`：在终端点中进入该模型的流量百分比，指定为一个或多个键/值对的字典。
  - 如果只有一个模型，则指定为**{ "0": 100 }**，其中"0"指的是要上传的这个模型，100表示100%的流量。
  - 如果在终端点上存在之前的模型，流量将被分配，则使用`model_id`指定为**{ "0": percent, model_id: percent, ... }**，其中`model_id`是要部署至已部署终端点的现有模型的模型标识。这些百分比必须加起来等于100。

现在让我们深入了解`deployed_model`参数。该参数被指定为Python字典，具有最少所需的字段：

- `model`：要部署的 Vertex 完全限定模型标识符（上传）模型。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：禁用容器事件的记录，例如执行失败（默认情况下启用容器日志记录）。通常在调试部署时启用容器日志记录，然后在生产部署时禁用。
- `dedicated_resources`：为提供预测请求而扩展的计算实例（副本）数量。
  - `machine_spec`：要提供的计算实例。使用之前设置的变量`DEPLOY_GPU != None`以使用GPU；否则只分配一个CPU。
  - `min_replica_count`：最初要提供的计算实例数量，您将其设置为变量`MIN_NODES`。
  - `max_replica_count`：要扩展到的计算实例的最大数量，您将其设置为变量`MAX_NODES`。

#### 流量分配

现在让我们深入了解`traffic_split`参数。该参数被指定为Python字典。这可能起初有点混淆。让我解释一下，您可以将您的模型的更多实例部署到一个终端点，然后设置每个实例接收多少流量（百分比）。

为什么要这样做呢？也许您已经在生产环境中部署了以前的版本 -- 让我们称之为 v1。您在 v2 上获得了更好的模型评估，但在部署到生产环境之前，您还不确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的终端点，但它只占总流量的10%。这样，您可以在不干扰大部分用户的情况下监视它的表现 -- 直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作`response`。我们将通过调用`response.result()`同步等待操作完成，这将阻塞直到模型部署完成。如果这是第一次将模型部署到终端点，可能需要额外几分钟来完成资源的部署。

In [ ]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

进行在线预测请求

现在对部署的模型进行在线预测。

获取测试项

您将使用数据集中的测试（保留）部分的示例作为测试项目。

In [ ]:
test_image = x_test_lr[0]
test_label = y_test[0]
print(test_image.shape)

### 发送预测请求

好的，现在您有一个测试图片。使用这个辅助函数`predict_image`，它接受以下参数：

- `image`：作为numpy数组的测试图片数据。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的Vertex完全限定标识符。
- `parameters_dict`：用于serving的额外参数。

此函数使用以下参数调用预测客户端服务`predict`方法：

- `endpoint`：`Model`资源部署到的`Endpoint`资源的Vertex完全限定标识符。
- `instances`：要预测的实例（编码图像）的列表。
- `parameters`：用于serving的额外参数。

要将图片数据传递到预测服务，在之前的步骤中，您将字节编码为base64 -- 这样在通过网络传输二进制数据时可以确保内容不会被修改。您需要告诉serving二进制文件您的模型部署在哪里，内容已经被base64编码，以便它在上传到serving二进制文件时进行解码。

预测请求中的每个实例是一个形如：

                        {serving_input: {'b64': content}}

的字典条目。

- `input_name`：底层模型的输入层的名称。
- `'b64'`：指示内容是base64编码的键。
- `content`：压缩后的JPG图像字节作为base64编码的字符串。

由于`predict()`服务可以处理多个图片（实例），您将把单个图片作为一个图片列表发送。最后一步，您将实例列表打包成谷歌的protobuf格式 -- 这是我们传递给`predict()`服务的内容。

`response`对象返回一个列表，列表中的每个元素分别对应请求中的每个图片。您将在每个预测的输出中看到：

- `predictions`：每个类别的预测置信度，介于0和1之间。

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters_dict
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_image(b64str, endpoint_id, None)

## 取消部署 `Model` 资源

现在请从服务 `Endpoint` 资源中取消部署您的 `Model` 资源。使用以下辅助函数 `undeploy_model`，该函数接受以下参数：

- `deployed_model_id`：在将 `Model` 资源部署到服务时由端点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 Vertex 完全限定标识符所在的 `Endpoint` 资源。

此函数调用端点客户端服务的方法 `undeploy_model`，并提供以下参数：

- `deployed_model_id`：端点服务在部署 `Model` 资源时返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 Vertex 完全限定标识符所在的 `Endpoint` 资源。
- `traffic_split`：如何在 `Endpoint` 资源上剩余部署的模型之间分配流量。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以简单地通过将其设置为空对象 {} 来使 `traffic_split` 为空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的以下个别资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME